<a href="https://colab.research.google.com/github/AashiDutt/AI-and-ML-for-Coders/blob/main/Chapter_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer 

In [2]:
sentences= [
            'Today is a sunny day',
            'Today is a rainy day',
            'Is it sunny today?', #tokenizer removes punctuation
            'I really enjoyed walking in the snow today'
            
]

tokenizer = Tokenizer(num_words = 100) #specify no. of words tokenizer can tokenize
tokenizer.fit_on_texts(sentences) #creates tokenized word index
word_index = tokenizer.word_index
print(word_index)

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7, 'i': 8, 'really': 9, 'enjoyed': 10, 'walking': 11, 'in': 12, 'the': 13, 'snow': 14}


In [3]:
#converting sentences in list of numbers
# numbers - value
# word - key

sequences = tokenizer.texts_to_sequences(sentences)

print(sequences)

[[1, 2, 3, 4, 5], [1, 2, 3, 6, 5], [2, 7, 4, 1], [8, 9, 10, 11, 12, 13, 14, 1]]


"Today is a sunny day" is encoded as  - [1, 2, 3, 4, 5]

"Today is a rainy day" is encoded as - [1, 2, 3, 6, 5]

"Is it sunny today?" is encoded as - [2, 7, 4, 1]

In [4]:
#for previously unseen data

test_data = [
             'Today is a snowy day',
             'Will it be rainy tomorrow?'
]

test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7, 'i': 8, 'really': 9, 'enjoyed': 10, 'walking': 11, 'in': 12, 'the': 13, 'snow': 14}
[[1, 2, 3, 5], [7, 6]]


Swapping back tokens 

[7,6] - it rainy (Actual - will it be rainy tomorrow?)

In [5]:
#Out of vocabulary token 
tokenizer = Tokenizer(num_words = 100, oov_token = "<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)

{'<OOV>': 1, 'today': 2, 'is': 3, 'a': 4, 'sunny': 5, 'day': 6, 'rainy': 7, 'it': 8, 'i': 9, 'really': 10, 'enjoyed': 11, 'walking': 12, 'in': 13, 'the': 14, 'snow': 15}
[[2, 3, 4, 1, 6], [1, 8, 1, 7, 1]]


Reverse encoding now will give 

[2,3,4,1,6] - today is a <OOV> day

[1,8,1,7,1] - <OOV> it <OOV> rainy <OOV> 

Most of words of 2nd sentence were not in corpus, thus it lacks context but it's a step in the right direction.

In [7]:
from os import truncate
#Different sentences = different sizes(length)
#Padding helps to get all sentences in same shape and size

#from tensorflow.keras.preprocessing.sequence
#import pad_sequences
#padded = pad_sequences(sequences)

padded = tf.keras.preprocessing.sequence.pad_sequences(sequences,padding = 'post', truncating='post')
print(padded)


[[ 2  3  4  5  6  0  0  0]
 [ 2  3  4  7  6  0  0  0]
 [ 3  8  5  2  0  0  0  0]
 [ 9 10 11 12 13 14 15  2]]


IMDb Dataset

In [8]:
import tensorflow_datasets as tfds

In [9]:
#removing stopwords that are too common and repeated
from bs4 import BeautifulSoup
import string 

stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

#removing Punctuations -otherwise, a stopword followed by comma won't be spotted
table = str.maketrans('','',string.punctuation)

#Get the Data
imdb_sentences = []
train_data = tfds.as_numpy(tfds.load('imdb_reviews', split = 'train'))

for item in train_data:
  sentence = str(item['text'].decode('UTF-8').lower()) #loading data as strings

#adding space around punctuation (him/her becomes him / her) and can be removed easily.

  sentence = sentence.replace(",", " , ") 
  sentence = sentence.replace(".", " . ")
  sentence = sentence.replace("-", " - ")
  sentence = sentence.replace("/", " / ")
  soup = BeautifulSoup(sentence)
  sentence = soup.get_text()
  words = sentence.split()
  filtered_sentence = ""
  for word in words:
    word = word.translate(table)
    if word not in stopwords:
      filtered_sentence = filtered_sentence + word + " "
  imdb_sentences.append(filtered_sentence)
  

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete308LCT/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete308LCT/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [10]:
#Assigning Tokens
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 25000)
tokenizer.fit_on_texts(imdb_sentences)
sequences = tokenizer.texts_to_sequences(imdb_sentences)
print(tokenizer.word_index)

{'movie': 1, 'film': 2, 'not': 3, 'one': 4, 'like': 5, 'just': 6, 'good': 7, 'no': 8, 'time': 9, 'even': 10, 'story': 11, 'really': 12, 'see': 13, 'can': 14, 'much': 15, 'bad': 16, 'get': 17, 'will': 18, 'also': 19, 'people': 20, 'great': 21, 'first': 22, 'dont': 23, 'made': 24, 'movies': 25, 'make': 26, 'way': 27, 'films': 28, 'characters': 29, 'think': 30, 'watch': 31, 'two': 32, 'character': 33, 'many': 34, 'seen': 35, 'life': 36, 'plot': 37, 'never': 38, 'acting': 39, 'little': 40, 'love': 41, 'best': 42, 'show': 43, 'know': 44, 'off': 45, 'ever': 46, 'better': 47, 'end': 48, 'still': 49, 'man': 50, 'say': 51, 'scene': 52, 'scenes': 53, 'go': 54, 'something': 55, 'back': 56, 'real': 57, 'watching': 58, 'years': 59, 'now': 60, 'doesnt': 61, 'though': 62, 'actors': 63, 'old': 64, 'thing': 65, 'didnt': 66, 'work': 67, 'new': 68, 'another': 69, 'nothing': 70, 'funny': 71, 'actually': 72, '10': 73, 'makes': 74, 'director': 75, 'look': 76, 'find': 77, 'going': 78, 'part': 79, 'every': 80

In [12]:
#Reverse encoding - sequence to sentence

reverse_word_index = dict(
    [(value,key) for (key,value) in
     tokenizer.word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i, '?')
for i in sequences[0]])

print(decoded_review)

absolutely terrible movie dont lured christopher walken michael ironside great actors must simply worst role history even great acting not redeem movies ridiculous storyline movie early nineties us propaganda piece pathetic scenes rebels making cases revolutions maria conchita alonso appeared phony pseudo love affair walken nothing pathetic emotional plug movie devoid real meaning disappointed movies like ruining actors like christopher walkens good name barely sit


IMDB Subwords dataset

In [13]:
(train_data, test_data), info = tfds.load('imdb_reviews/subwords8k',
                                          split=(tfds.Split.TRAIN, tfds.Split.TEST),
                                          as_supervised = True,
                                          with_info = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteC7VDSQ/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteC7VDSQ/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [14]:
encoder = info.features['text'].encoder
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [15]:
print(encoder.subwords)

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br', 'in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 'as_', 't_', 'with_', 'for_', '.<', 'on_', 'but_', 'movie_', ' (', 'are_', 'his_', 'have_', 'film_', 'not_', 'ing_', 'be_', 'ed_', 'you_', ' "', 'it', 'd_', 'an_', 'he_', 'by_', 'at_', 'one_', 'who_', 'y_', 'from_', 'e_', 'or_', 'all_', 'like_', 'they_', '" ', 'so_', 'just_', 'has_', ') ', 'her_', 'about_', 'out_', 'This_', 'some_', 'ly_', 'movie', 'film', 'very_', 'more_', 'It_', 'would_', 'what_', 'when_', 'which_', 'good_', 'if_', 'up_', 'only_', 'even_', 'their_', 'had_', 'really_', 'my_', 'can_', 'no_', 'were_', 'see_', 'she_', '? ', 'than_', '! ', 'there_', 'get_', 'been_', 'into_', ' - ', 'will_', 'much_', 'story_', 'because_', 'ing', 'time_', 'n_', 'we_', 'ed', 'me_', ': ', 'most_', 'other_', 'don', 'do_', 'm_', 'es_', 'how_', 'also_', 'make_', 'its_', 'could_', 'first_', 'any_', "' ", 'people_', 'great_', 've_', 'ly', 'er_', 'made_', 'r_', 'B

In [20]:
sample_string = "Today is a sunny day"
encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

Encoded string is [6427, 4869, 9, 4, 2365, 1361, 606]


In [22]:
print(encoder.subwords[6426]) #addressing an array

Tod


In [23]:
#decoding

orignal_string = encoder.decode(encoded_string)
test_string = encoder.decode([6427, 4869, 9, 4, 2365, 1361, 606])

print(test_string)

Today is a sunny day
